<a href="https://colab.research.google.com/github/OskarKrafft/Machine-Learning-Project/blob/main/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries

In [ ]:
# Install packages and import them

In [ ]:
import pandas as pd
import pyreadstat

## Set up WD

In [ ]:
# Mount GDrive to facilitate data import
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Change working directory to project folder
%cd /content/drive/MyDrive/Colab Notebooks/Machine-Learning-Project

In [ ]:
# If Github repo is not yet cloned in Drive, use following code (just do once!):

# ! git clone https://github.com/OskarKrafft/Machine-Learning-Project.git

## Import data

In [ ]:
eppes, meta = pyreadstat.read_sav('data/raw/1.0-EPPES.sav')
pd.set_option('display.max_columns', None)
eppes

# Data Cleaning

## Drop non-voters

In [ ]:
# Drop observation of people that were not eligible to vote

eppes = eppes[eppes.qg1.isna() == False]
eppes

In [ ]:
# Check for NAs in vote variable qg1

eppes['qg1'].isna().sum()

In [ ]:
# Create frequency table of qg1

eppes["qg1"].value_counts()

## Drop problematic, non-complete or uninteresting columns

In [ ]:
# Import Excel sheet containing column indeces to be dropped

columns_analysis = pd.read_excel(r'Drop_Columns.xlsx')
columns_analysis = columns_analysis.drop(columns_analysis.columns[[0]], axis = 1)
print(columns_analysis)

In [ ]:
# Create list of indeces of columns to be dropped

indeces_to_drop = []

for i in range(872):
  if columns_analysis.iloc[i, 2] == False:
    indeces_to_drop.append(i)

print(indeces_to_drop)

In [ ]:
# Drop columns by indeces_to_drop

eppes = eppes.drop(eppes.columns[indeces_to_drop], axis = 1)
eppes

## Deal with NAs

In [ ]:
# Check for NAs in columns
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
  print(eppes.isna().sum())

Result: qc2a_1 and qc2a_2, d1, d25, d40b contain NAs

Next Steps

1. **qc2a_1** and **qc2a_2**: 

For each of the following statements, please tell me whether you totally agree, tend to agree, tend to disagree or totally disagree.

qc2a_1: Measures to reduce the public deficit and debt in (OUR COUNTRY) cannot be delayed

qc2a_2: Measures to reduce the public deficit and debt in (OUR COUNTRY) are not a priority for now

For these two questions, the dataset was split into two groups (former columns index 12) and only group 1 was asked qc2a_1, only group 2 was asked qc2a_2

-> Drop qc2a_1 and qc2a_2 columns

2. **d1**: 

In political matters people talk of "the left" and "the right". How would you place your views on this scale?

Important Question

-> Change values of rows with d1 == NA to d1 == 12 (DK)

3. **d25**:

Would you say you live in a...? Rural area or village, small or middle sized town, etc.

-> Change values of rows with d25 == NA to d25 == 4 (DK)

4. **d40b**:

Could you tell me how many children less than 10 years old live in your household?

-> replace values with 0

In [ ]:
# Drop qc2a_1 and qc2a_2

eppes = eppes.drop(['qc2a_1', 'qc2a_2'], axis=1)

# Replace values of rows with d1 == NA with d1 == 12

eppes.loc[eppes['d1'].isna(), 'd1'] = 12

# Replace values of rows with d25 == NA with d25 == 4

eppes.loc[eppes['d25'].isna(), 'd25'] = 4

# Replace values of rows with d40b == NA with d40b == 0

eppes.loc[eppes['d40b'].isna(), 'd40b'] = 0


## warum nicht die NAs bei d40b direkt droppen? sind im endeffekt ja auch nur 2 observations.

In [ ]:
# Check for NAs in eppes
eppes.isna().sum().sum()

## Delete rows with NAs in ordinal variables

In [ ]:
eppes.describe()

d1, d7 and d11 have values of 97/98?

In [ ]:
eppes["d1"].value_counts()

In [ ]:
eppes["d7"].value_counts()

In [ ]:
eppes["d11"].value_counts()

Explanation: 

- d1 and d7: probably DKs
- d11: true age, no changes necessary
- for most variables: Highest number stands for DK


In [ ]:
for i in range(86):
  print(eppes[col_names_ordinal[i]].value_counts())

In [ ]:
columns_analysis["encoding"].value_counts()

In [ ]:
# Create list of names of ordinal columns 

col_names_ordinal = []

for i in range(872):
  if columns_analysis.iloc[i, 3] == 'ordinal':
    col_names_ordinal.append(columns_analysis.iloc[i, 1])

# Delete qa1a_1 and qa1a_2 see above
col_names_ordinal.remove('qc2a_1')
col_names_ordinal.remove('qc2a_2')

print(col_names_ordinal)

In [ ]:
# Drop DKs for ordinal variables

for i in range(86):
  eppes = eppes[eppes[col_names_ordinal[i]] != max(eppes[col_names_ordinal[i]])]


In [ ]:
for i in range(86):
  print(eppes[col_names_ordinal[i]].value_counts())

In [ ]:
eppes.describe()

In [ ]:
# Data Exploration

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(eppes.dtypes)

## Change Value type from float to object for categoricals

In [ ]:
eppes.dtypes.value_counts()

In [ ]:
# Create list of names of categorical columns 

col_names_categorical = []

for i in range(872):
  if columns_analysis.iloc[i, 3] == 'categorical':
    col_names_categorical.append(columns_analysis.iloc[i, 1])

In [ ]:
# Change datatype of categorical variables to object

eppes_clean = eppes

eppes_clean[col_names_categorical] = eppes_clean[col_names_categorical].astype('object')

eppes_clean.dtypes.value_counts()